In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta

In [2]:
# the measurement station codes (35 stations)
# more detailed information on the stations can be found in metadata/measurement_sites_info.txt
stations = ['ABZ', 'ALE', 'AMA', 'AMM', 'ASP', 'BEI', 'BOT', 'BSL', 'DEL', 'EGB',
            'FKL', 'HAD', 'HEL', 'HPB', 'HRW', 'HYY', 'KCE', 'KPZ', 'MAR', 'MHD', 
            'MLP', 'MUK', 'NAN', 'NEU', 'POV', 'SAO', 'SCH', 'SGP', 'UAE', 'PRL',
            'VAR', 'VHL', 'VIE', 'WAL', 'ZOT']

In [3]:
def combine_datasets(folder):
    '''
    Concatenates the individual datasets for all measurement stations in a specified folder located in "data/".
    
    Parameters:
    folder (str): Name of folder located in the "data/" folder of the repository

    Returns:
    pd.DataFrame: data frame of the concatenated data sets in the folder
    '''
    
    full_df = []

    for s in stations:
        df = pd.read_csv('data/'+folder+'/'+s+'.csv')
        df['station'] = s
        full_df.append(df)

    full_df = pd.concat(full_df)
    full_df = full_df.reset_index(drop=True)
    full_df['id'] = [full_df.station[i] + '-' + str(full_df.date[i]) for i in range(full_df.shape[0])]
    
    return full_df

In [4]:
# loading the aerosol mixing ratio data and saving it as a separate csv file
# a full description of all variable names can be found in metadata/variable_names.txt
aerosols_df = combine_datasets('aerosols')
aerosols_df = aerosols_df[['id', 'latitude', 'longitude', 'aermr01', 'aermr02', 
                           'aermr03', 'aermr04', 'aermr05', 'aermr06', 'aermr07', 
                           'aermr08', 'aermr09', 'aermr10', 'aermr11']]
aerosols_df.to_csv('data/aerosols_data.csv', index=False)

In [5]:
# loading the atmospheric data, calculating the relative humidity, and saving it as a separate csv file
atmospheric_df = combine_datasets('atmospheric')
atmospheric_df = atmospheric_df[['id', 'd2m', 't2m']]
td = atmospheric_df.d2m - 273.15
t = atmospheric_df.t2m - 273.15
atmospheric_df['rh'] = 100*(np.exp((17.625*td) / (243.04+td)) / np.exp((17.625*t) / (243.04+t)))
atmospheric_df.to_csv('data/atmospheric_data.csv', index=False)

In [6]:
# loading the boundary layer height data and saving it as a separate csv file
boundary_layer_height_df = combine_datasets('boundary_layer_height')
boundary_layer_height_df = boundary_layer_height_df[['id', 'blh']]
boundary_layer_height_df.to_csv('data/boundary_layer_height_data.csv', index=False)

In [7]:
# loading the gas data and saving it as a separate csv file
gases_df = combine_datasets('gases')
gases_df = gases_df[['id', 'co', 'c5h8', 'no2', 'no', 'so2']]
gases_df.to_csv('data/gases_data.csv', index=False)

In [8]:
# loading the slow access data and saving it as a separate csv file
slow_access_df = combine_datasets('slow_access')
slow_access_df = slow_access_df[['id', 'nh3', 'crwc', 'c10h16']]
slow_access_df.to_csv('data/slow_access_data.csv', index=False)

In [9]:
# loading the wind data, calculating the wind speed, and saving it as a separate csv file
wind_df = combine_datasets('wind')
wind_df['wind_speed'] = np.sqrt(wind_df.u**2 + wind_df.v**2)
wind_df = wind_df[['id', 'wind_speed']]
wind_df.to_csv('data/wind.csv', index=False)

In [10]:
# loading the n100 data and saving it as a separate csv file
n100_df = []
for s in stations:
    df = pd.read_table('data/N100_proxy/'+s+'_N100.dat', sep='\s+', 
                       names=['year', 'month', 'day', 'hour', 'minute', 'n100'])
    
    # AMA and UAE are in UTC, so the times of the measurements need to be adjusted to local time
    if (s == 'AMA'):
        df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
        df.date -= timedelta(hours=4)
        df.date = [d.replace(hour=0, minute=0, second=0) for d in df.date]      
    elif (s == 'UAE'):
        df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
        df.date += timedelta(hours=4)
        df.date = [d.replace(hour=0, minute=0, second=0) for d in df.date]
        
    # the year for FKL is multiplied by 2 due to some bug, so needs to be corrected
    elif (s == 'FKL'):
        df.year /= 2
        df['date'] = pd.to_datetime(df[['year', 'month', 'day']])  
        
    else:
        df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
        
    df = df.groupby('date', as_index=False).mean()
    df.date = df.date.dt.date
    df['station'] = s
    n100_df.append(df)

n100_df = pd.concat(n100_df)
n100_df = n100_df.reset_index(drop=True)
n100_df['id'] = [n100_df.station[i] + '-' + str(n100_df.date[i]) for i in range(n100_df.shape[0])]
n100_df = n100_df[['id', 'station', 'date', 'n100']]
n100_df.to_csv('data/n100_data.csv', index=False)

In [11]:
# combining all the data sets into one dataframe
data = n100_df.merge(aerosols_df, on='id')
data = data.merge(atmospheric_df, on='id')
data = data.merge(boundary_layer_height_df, on='id')
data = data.merge(gases_df, on='id')
data = data.merge(slow_access_df, on='id')
data = data.merge(wind_df, on='id')

# calculating the day of the year (doy) adding the sine and cosine of the doy to the dataframe
data['date'] =  pd.to_datetime(data['date'])
doy = data.date.dt.dayofyear
data['doy_sin'] = np.sin((2*np.pi*doy)/doy.max())
data['doy_cos'] = np.cos((2*np.pi*doy)/doy.max())

# removing outliers for VHL
for i, row in data.loc[data.station == 'VHL'].iterrows():
    if row.n100 > 10000:
        data.drop(index=i, inplace=True)

# reordering the columns
data = data[['id', 'station', 'date', 'doy_sin', 'doy_cos', 'latitude', 'longitude', 'n100', 
             'aermr01', 'aermr02', 'aermr03', 'aermr04', 'aermr05', 'aermr06', 'aermr07', 
             'aermr08', 'aermr09', 'aermr10', 'aermr11', 'co', 'c5h8', 'c10h16', 'nh3', 
             'no', 'no2', 'so2', 'd2m', 't2m', 'crwc', 'blh', 'rh', 'wind_speed']]

# saving the full dataset
data.date = pd.to_datetime(data.date)
data = data.dropna(axis=0)
data.to_csv('data/full_data.csv', index=False)
data.head()

,id,station,date,doy_sin,doy_cos,latitude,longitude,n100,aermr01,aermr02,...,nh3,no,no2,so2,d2m,t2m,crwc,blh,rh,wind_speed
0,ABZ-2012-01-26,ABZ,2012-01-26,0.431673,0.902030,50.57,12.99,2708.085714,6.752697e-12,5.765684e-10,...,8.448101e-10,1.302913e-08,2.060121e-08,6.464830e-09,266.65186,268.37625,0.0,306.51462,87.716470,1.228889
1,ABZ-2012-01-27,ABZ,2012-01-27,0.447094,0.894487,50.57,12.99,2489.175000,4.223335e-12,3.600730e-10,...,1.008734e-09,1.489063e-08,2.199851e-08,7.113044e-09,265.71246,267.83536,0.0,142.03300,85.013445,0.962691
2,ABZ-2012-01-28,ABZ,2012-01-28,0.462383,0.886680,50.57,12.99,3484.229167,3.553853e-12,3.007895e-10,...,1.407371e-09,3.310498e-08,1.887175e-08,7.495814e-09,268.29993,269.69257,0.0,152.80160,90.074210,0.785850
3,ABZ-2012-01-29,ABZ,2012-01-29,0.477536,0.878612,50.57,12.99,2977.512500,8.665312e-12,7.307493e-10,...,1.466395e-09,4.424891e-09,1.569601e-08,4.929475e-09,267.07495,269.08804,0.0,461.47455,85.875228,1.748744
4,ABZ-2012-01-30,ABZ,2012-01-30,0.492548,0.870285,50.57,12.99,2702.463636,1.436837e-11,1.208825e-09,...,1.477080e-09,1.021235e-09,1.345031e-08,5.219232e-09,263.61273,267.95770,0.0,610.29910,71.525093,2.068425
